In [65]:
# Install a pip package in the current Jupyter kernel
!python --version
import sys
!{sys.executable} -m pip install numpy pandas matplotlib

Python 3.11.9


In [66]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [67]:
def resultdir(resultsize, dbsize, index):
    return f"results_N{resultsize}_D{dbsize}_I{index}"

def calculate_mean(q, method,table, resultsize, dbsize, index):
    global resultdirs
    resdir = resultdir(resultsize, dbsize, index)
    #url = f'https://raw.githubusercontent.com/lordpretzel/prov_semanticopt_experiment/master/{resdir}/runtime_{q}_{table}_{method}.csv'
    url = f"./{resdir}/runtime_{q}_{table}_{method}.csv"
    df = pd.read_csv(url, header=None, delimiter=',')
    df = df.set_axis(["time"], axis=1)
    return df['time'].mean()

allmethods = ['flatten', 'unopt', 'unoptheu', 'opt', 'optnoflat', 'optheu']
allqueries = ['01','02','03', '04', '05', '06', '07', '08', '09','10','11', '14', '17', '19', '20'] # ,'02','03']
resultsizes = [ '1', '10p', '25p', '100p' ] #, '10p', '25p', '100p' ]
dbsizes = [ '1GB','10GB' ]    
index = [ 'noindex', 'index' ]
# resultdirs = { x: "results_n1" for x in  ['flatten', 'unopt', 'opt', 'optnoflat'] }
# resultdirs |= { x: "testopt_results" for x in ['unoptheu', 'optheu'] }

queries = {
    "01": ["lineitem"],
    "02": ["nation", "part", "partsupp", "region"],
    "03": ["customer", "lineitem", "orders"],
    "04": ["lineitem"], #"orders"],
    "05": ["customer", "lineitem", "nation", "orders", "region"],
    "06": ["lineitem"],
    "07": ["customer", "lineitem", "nation", "orders", "supplier"],
    "08": ["customer", "nation", "part", "supplier"],
    "09": ["lineitem", "nation", "orders", "part", "partsupp", "supplier"],
    "10": ["customer", "lineitem", "nation", "orders"],
    "11": ["nation", "partsupp", "supplier"],
    "12": ["lineitem", "orders"],
    "13": ["customer", "orders"],
    "14": ["lineitem", "part"],
    "15": ["lineitem", "supplier"],
    "16": ["part", "partsupp", "supplier"],
    "17": ["lineitem", "part"],
    "18": ["customer", "lineitem", "orders"],
    "19": ["lineitem", "part"],
    "20": ["part"],
    }

In [68]:
allresult = {}
for d in dbsizes:
    allresult[d] = {}
    for i in index:
        allresult[d][i] = {}
        for n in resultsizes:
            allresult[d][i][n] = {}
            for q in allqueries: #['01','02', ...
                allresult[d][i][n][q] = {} #has to exist before assigning sub-whatever
                for table in queries[q]: #[part,region, ...]
                    resultsforq = {}
                    for method in allmethods: # ['flatten', 'unopt', 'opt'] 
                        try:
                            mean = calculate_mean(q,method,table,n,d,i)
                            resultsforq[method] = mean
                            #print(resultsforq)
                        except Exception as e:
                            print(f"ERROR DB: {d} [{i}] with numresults {n} for {q} for {table} using {method}: {e}")
                #print(allresult)
                    allresult[d][i][n][q][table] = resultsforq
allresult 

ERROR DB: 1GB [noindex] with numresults 25p for 01 for lineitem using flatten: [Errno 2] No such file or directory: './results_N25p_D1GB_Inoindex/runtime_01_lineitem_flatten.csv'
ERROR DB: 1GB [noindex] with numresults 25p for 01 for lineitem using unopt: [Errno 2] No such file or directory: './results_N25p_D1GB_Inoindex/runtime_01_lineitem_unopt.csv'
ERROR DB: 1GB [noindex] with numresults 25p for 01 for lineitem using unoptheu: [Errno 2] No such file or directory: './results_N25p_D1GB_Inoindex/runtime_01_lineitem_unoptheu.csv'
ERROR DB: 1GB [noindex] with numresults 25p for 01 for lineitem using opt: [Errno 2] No such file or directory: './results_N25p_D1GB_Inoindex/runtime_01_lineitem_opt.csv'
ERROR DB: 1GB [noindex] with numresults 25p for 01 for lineitem using optnoflat: [Errno 2] No such file or directory: './results_N25p_D1GB_Inoindex/runtime_01_lineitem_optnoflat.csv'
ERROR DB: 1GB [noindex] with numresults 25p for 01 for lineitem using optheu: [Errno 2] No such file or directo

{'1GB': {'noindex': {'1': {'01': {'lineitem': {'flatten': 11372.6875,
      'unopt': 11564.1845,
      'unoptheu': 11578.09505,
      'opt': 11377.908200000002,
      'optnoflat': 11570.60455,
      'optheu': 11575.2851}},
    '02': {'nation': {'flatten': 53542.186499999996,
      'unopt': 56.25590000000001,
      'unoptheu': 51.693949999999994,
      'opt': 38.95895,
      'optnoflat': 47.22565,
      'optheu': nan},
     'part': {'flatten': 54134.25134999999,
      'unopt': 50.86395,
      'unoptheu': 51.229600000000005,
      'opt': 8.991699999999998,
      'optnoflat': 44.9356,
      'optheu': nan},
     'partsupp': {'flatten': 55772.769349999995,
      'unopt': 51.5404,
      'unoptheu': 50.519149999999996,
      'opt': 38.77460000000001,
      'optnoflat': 46.32035,
      'optheu': nan},
     'region': {'flatten': 51600.72110000001,
      'unopt': 52.551700000000004,
      'unoptheu': 50.17659999999999,
      'opt': 38.88425,
      'optnoflat': 46.73995,
      'optheu': nan}},
  

In [69]:
def dfs_for_experiments(results, methods):
    edfs = {}
    for q in allqueries:
        qdata = results[q]
        qdict = { k: [] for k in usemethods } # allmethods }
        tables = qdata.keys()
        for t in qdata:
            for m in usemethods:
                val = qdata[t][m] if m in qdata[t] else np.nan
                val = val if val > 0.9 else np.nan
                qdict[m] = qdict[m] + [val]
        edfs[q] = pd.DataFrame(data=qdict, index=tables)
        edfs[q] = round(edfs[q],2)
    return edfs
    
dfs = {}
usemethods=allmethods

for d in dbsizes:
    dfs[d] = {}
    for i in index:
        dfs[d][i] = {}
        for n in resultsizes:
            dfs[d][i][n] = {}
            dfs[d][i][n] = dfs_for_experiments(allresult[d][i][n], usemethods)

dfs

{'1GB': {'noindex': {'1': {'01':            flatten     unopt  unoptheu       opt  optnoflat    optheu
    lineitem  11372.69  11564.18   11578.1  11377.91    11570.6  11575.29,
    '02':            flatten  unopt  unoptheu    opt  optnoflat  optheu
    nation    53542.19  56.26     51.69  38.96      47.23     NaN
    part      54134.25  50.86     51.23   8.99      44.94     NaN
    partsupp  55772.77  51.54     50.52  38.77      46.32     NaN
    region    51600.72  52.55     50.18  38.88      46.74     NaN,
    '03':           flatten  unopt  unoptheu   opt  optnoflat  optheu
    customer     4.95   5.21      4.75  4.09       4.37    4.05
    lineitem     5.03   5.16      4.92  3.17       3.37    3.14
    orders       4.89   5.06      4.70  3.02       3.26    3.03,
    '04':           flatten    unopt  unoptheu    opt  optnoflat  optheu
    lineitem   443.01  3473.84    440.69  440.9    3399.37  441.43,
    '05':           flatten   unopt  unoptheu     opt  optnoflat  optheu
    cust

In [70]:
dfs['1GB']['index']['1']

{'01':            flatten     unopt  unoptheu       opt  optnoflat    optheu
 lineitem  11428.14  11665.23  11469.79  11444.47   11639.03  11522.66,
 '02':           flatten  unopt  unoptheu     opt  optnoflat  optheu
 nation    1070.77  48.71     48.35  868.61      43.90     NaN
 part       994.22  48.88     51.84    9.93      45.38     NaN
 partsupp  1099.96  51.40     48.56  911.44      44.76     NaN
 region    1001.17  48.56     51.57  869.43      45.83     NaN,
 '03':           flatten  unopt  unoptheu   opt  optnoflat  optheu
 customer     5.02   5.33      4.81  3.98       4.23    3.90
 lineitem     5.04   5.20      4.94  3.01       3.20    2.95
 orders       4.96   5.12      4.78  2.86       3.06    2.83,
 '04':           flatten   unopt  unoptheu     opt  optnoflat  optheu
 lineitem   450.68  3454.2    447.29  443.51    3391.49  444.44,
 '05':           flatten   unopt  unoptheu     opt  optnoflat  optheu
 customer   272.70  278.39    276.79  272.37     272.66  273.48
 lineitem

In [71]:
def best_opt_noopt_for_experiemnt(edfs):
    ebestdfs = {}
    for q in allqueries:
        df = edfs[q].copy()
        df = df.fillna(1000000.0)        
        df['nonopt-best'] = df[['flatten','unopt','unoptheu']].min(axis=1)
        df['opt-best'] = df[['opt','optnoflat','optheu']].min(axis=1) 
        df = df.drop(columns=['opt','optnoflat','flatten','unopt'])
        cols = ['nonopt-best', 'opt-best']
        df = df[cols]
        ebestdfs[q] = df
    return ebestdfs

bestdfs = {}
for d in dbsizes:
    bestdfs[d] = {}
    for i in index:
        bestdfs[d][i] = {}
        for n in resultsizes:
            bestdfs[d][i][n] = {}
            bestdfs[d][i][n] = best_opt_noopt_for_experiemnt(dfs[d][i][n])

bestdfs

{'1GB': {'noindex': {'1': {'01':           nonopt-best  opt-best
    lineitem     11372.69  11377.91,
    '02':           nonopt-best  opt-best
    nation          51.69     38.96
    part            50.86      8.99
    partsupp        50.52     38.77
    region          50.18     38.88,
    '03':           nonopt-best  opt-best
    customer         4.75      4.05
    lineitem         4.92      3.14
    orders           4.70      3.02,
    '04':           nonopt-best  opt-best
    lineitem       440.69     440.9,
    '05':           nonopt-best  opt-best
    customer       231.19    231.70
    lineitem       229.57    230.11
    nation         229.54    229.52
    orders         227.66    228.92
    region         228.06    227.99,
    '06':           nonopt-best  opt-best
    lineitem      1144.23   1149.21,
    '07':           nonopt-best  opt-best
    customer      3428.92   3066.93
    lineitem      3371.98   2960.19
    nation       10914.08  20758.28
    orders        3394.44   3

In [72]:
bestdfs['1GB']['index']['1']

{'01':           nonopt-best  opt-best
 lineitem     11428.14  11444.47,
 '02':           nonopt-best  opt-best
 nation          48.35     43.90
 part            48.88      9.93
 partsupp        48.56     44.76
 region          48.56     45.83,
 '03':           nonopt-best  opt-best
 customer         4.81      3.90
 lineitem         4.94      2.95
 orders           4.78      2.83,
 '04':           nonopt-best  opt-best
 lineitem       447.29    443.51,
 '05':           nonopt-best  opt-best
 customer       272.70    272.37
 lineitem       280.72    280.59
 nation         268.14    268.49
 orders         277.74    282.13
 region         267.18    276.02,
 '06':           nonopt-best  opt-best
 lineitem       1152.1   1151.94,
 '07':           nonopt-best  opt-best
 customer      2244.49   4273.88
 lineitem      2228.51   3967.69
 nation       11451.60  20719.39
 orders        3702.29  10550.51
 supplier      2265.09   3754.82,
 '08':           nonopt-best  opt-best
 customer       306.9

# All data together and compute stats

In [73]:
def plotquery(q, df, ax, show=False):
    tables = list(df.index)
    series = df.columns.to_list()
    numbars = len(series)
    x = np.arange(len(tables))  # the label locations
    width = 0.3  # the width of the bars

    rects = {}
    for i, m in enumerate(series):
        rects[m] = ax.bar(x - width/2 + (width/numbars)*i, df[m], width/4, label=m)
        ax.bar_label(rects[m], padding=3)

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Runtime (msec)')
    ax.set_title('Capture Runtime ' + q)
    ax.set_xticks(x, tables)
    ax.legend()
    if show:
        plt.show()

In [84]:
# ignoring q7 for now
#TODO needs to be fixed
merged_dfs = {}
relative_avgs = {}
total_dfs = {}
for d in dbsizes:
    relative_avgs[d] = {}
    total_dfs[d] = {}
    for i in index:
        relative_avgs[d][i] = {}
        total_dfs[d][i] = {}
        for n in resultsizes:
            relative_avgs[d][i][n] = {}
            print(f"db {d} index: {i}: resultsize: {n}")
            ddfs = dfs[d][i][n]
            merged_df = pd.concat([ ddfs[q] for q in ddfs.keys() if q != '07' ])
            merged_df['nonopt-best'] = merged_df[['flatten','unopt']].min(axis=1)
            merged_df['opt-best'] = merged_df[['opt','optnoflat']].min(axis=1) 
            merged_dfs[d] = merged_df
            
            total_df = merged_df[['nonopt-best','opt-best']].sum()
            relative = merged_df.copy()
            relative['min'] = relative.min(axis=1)
            for c in merged_df.columns:
                relative[c] = relative[c] / relative['min']

            total_dfs[d][i][n] = total_df
            relative_avg = relative.mean()    
            relative_avgs[d][i][n] = relative_avg            
            #print(total_df)
            print(relative_avg)

db 1GB index: noindex: resultsize: 1
flatten         263.543124
unopt             3.650341
unoptheu          3.829627
opt               1.215494
optnoflat         1.624965
optheu            1.065310
nonopt-best       3.192376
opt-best          1.166152
min            1262.184872
dtype: float64
db 1GB index: noindex: resultsize: 10p
flatten         149.352476
unopt             3.385173
unoptheu          3.905294
opt              73.663820
optnoflat         1.266108
optheu            1.071718
nonopt-best       3.172525
opt-best          1.118184
min            2169.433846
dtype: float64
db 1GB index: noindex: resultsize: 25p
flatten       NaN
unopt         NaN
unoptheu      NaN
opt           NaN
optnoflat     NaN
optheu        NaN
nonopt-best   NaN
opt-best      NaN
min           NaN
dtype: float64
db 1GB index: noindex: resultsize: 100p
flatten       NaN
unopt         NaN
unoptheu      NaN
opt           NaN
optnoflat     NaN
optheu        NaN
nonopt-best   NaN
opt-best      NaN
min     

In [83]:
need_avgs = {}
need_avgs['1GB-index'] = relative_avgs['1GB']['index']['1']
need_avgs['1GB-noindex'] = relative_avgs['1GB']['noindex']['1']
need_avgs['10GB-index'] =relative_avgs['10GB']['index']['1']
need_avgs['10GB-noindex'] =relative_avgs['10GB']['noindex']['1']
need_avgs

{'1GB-index': flatten          5.778671
 unopt            1.754119
 unoptheu         1.338045
 opt              2.835908
 optnoflat        1.593720
 optheu           1.123443
 nonopt-best      1.495758
 opt-best         1.245226
 min            873.290513
 dtype: float64,
 '1GB-noindex': flatten         263.543124
 unopt             3.650341
 unoptheu          3.829627
 opt               1.215494
 optnoflat         1.624965
 optheu            1.065310
 nonopt-best       3.192376
 opt-best          1.166152
 min            1262.184872
 dtype: float64,
 '10GB-index': flatten           49.590703
 unopt              2.766656
 unoptheu           2.350609
 opt                7.966185
 optnoflat          2.281063
 optheu             9.529233
 nonopt-best        2.502761
 opt-best           1.057680
 min            16342.675641
 dtype: float64,
 '10GB-noindex': flatten         4085.939229
 unopt             21.716618
 unoptheu          27.732123
 opt             1109.085957
 optnoflat         

In [85]:
need_totals = {}
need_totals['1GB-index'] = total_dfs['1GB']['index']['1']
need_totals['1GB-noindex'] = total_dfs['1GB']['noindex']['1']
need_totals['10GB-index'] =total_dfs['10GB']['index']['1']
need_totals['10GB-noindex'] =total_dfs['10GB']['noindex']['1']
need_totals

{'1GB-index': nonopt-best    44251.74
 opt-best       44138.97
 dtype: float64,
 '1GB-noindex': nonopt-best    62819.28
 opt-best       60648.15
 dtype: float64,
 '10GB-index': nonopt-best    698407.92
 opt-best       654487.92
 dtype: float64,
 '10GB-noindex': nonopt-best    633190.12
 opt-best       614035.37
 dtype: float64}

In [75]:
def experiment_plot(d,i,n,edfs,isbest):
    x, y = 4,4
    scale = 6
    plotfile = f'qplots_D{d}_N{n}_I{i}_{"best_for_opt_and_unopt" if isbest else "allmethods"}.pdf'
    plt.rcParams["figure.figsize"] = [scale * x, scale * y]
    fig, axs = plt.subplots(y,x)
    axs = [ item for a in axs for item in a ]
    for qpos, q in enumerate(allqueries):
        plotquery(q, edfs[q], axs[qpos])
    if os.path.exists(plotfile):
        os.remove(plotfile)
    plt.savefig(plotfile, format='pdf')
    plt.close()

os.chdir("./qplot/")
    
for d in dbsizes:
    for i in index:
        for n in resultsizes:
            for b in [ False, True ]:
                edfs = bestdfs[d][i][n] if b else dfs[d][i][n]
                experiment_plot(d,i,n, edfs,b)

print("created plots")                
os.chdir("..")

created plots


In [76]:
os.getcwd()

'/Users/lord_pretzel/Documents/Publications/2021_Murali-ProvenanceFiltering-With-Chase/prov_semanticopt_experiment'

In [77]:
print(os.getcwd())
#os.chdir("Publications/2021_Murali-ProvenanceFiltering-With-Chase/prov_semanticopt_experiment")
print(os.getcwd())

/Users/lord_pretzel/Documents/Publications/2021_Murali-ProvenanceFiltering-With-Chase/prov_semanticopt_experiment
/Users/lord_pretzel/Documents/Publications/2021_Murali-ProvenanceFiltering-With-Chase/prov_semanticopt_experiment
